In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

#### Carregando os dados

In [3]:
def check_features(df):
    for x in range(100): # loop de ,00 ,01 ,02...até 0,99 centavos
        if x >= 10:
            str_replace = ',' + str(x)
        else:    
            str_replace = ',0' + str(x)
        df['preco'] = df.preco.replace({str_replace:''}, regex=True)
    
    df['flex'] = pd.Series(dtype=int)
    
    for i in range(len(df)):
        # check preços
        df['preco'][i] = df['preco'][i].replace('.', '')
    
        # check flex
        if ('flex' in df['nome'][i].lower()):
            df['flex'][i] = 1
        else:
            df['flex'][i] = 0
        
        # check nomes
        if '(' in df['nome'][i]:
            df['nome'][i] = df['nome'][i][0:df['nome'][i].index("(")]
        else:
            df['nome'][i] = df['nome'][i][:-4]   
            
        # check ano
        df['ano'][i] = df['ano'][i][-4:]        
    
    df['preco'] = df['preco'].astype(float)
    
    return df

In [4]:
def load_df():    
    df = pd.read_csv('df_final.csv', sep='\t')
    df.drop(df.columns[[0, 1, 2]], axis = 1, inplace = True) 
    df = check_features(df)
    return df

In [5]:
df = load_df()
df.head(20)

<ipython-input-3-348df3b3d9d7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preco'][i] = df['preco'][i].replace('.', '')
<ipython-input-3-348df3b3d9d7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flex'][i] = 1
<ipython-input-3-348df3b3d9d7>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nome'][i] = df['nome'][i][0:df['nome'][i].index("(")]
<ipython-input-3-348df3b3d9d7>:28: SettingWithCopyWarning: 
A value is 

,nome,ano,km,cor,cambio,portas,preco,flex
0,Chevrolet Onix 1.0 LT,2020,0.000,Laranja,manual,3.0,59890.0,1.0
1,Fiat Strada Hard Working 1.4,2019,41.070,Branco,manual,2.0,45900.0,1.0
2,Fiat Strada Hard Working 1.4,2019,48.654,Prata,manual,4.0,45900.0,1.0
3,Fiat Strada Hard Working 1.4,2019,19.000,Prata,manual,2.0,45990.0,1.0
4,Fiat Strada Hard Working 1.4,2019,24.715,Branco,manual,2.0,45990.0,1.0
5,Chevrolet Onix 1.0 Turbo Premier,2020,0.000,Azul,automático,NaN,59900.0,0.0
6,Fiat Strada Hard Working 1.4,2019,39.441,Prata,manual,2.0,46000.0,1.0
7,Fiat Strada Hard Working 1.4,2019,18.860,Branco,manual,2.0,46300.0,1.0
8,Fiat Strada Hard Working 1.4,2019,40.856,Prata,manual,2.0,46500.0,1.0
9,Fiat Strada Freedom 1.4,2019,78.000,Vermelho,manual,NaN,46500.0,1.0


In [7]:
df.to_csv('df_teste.csv', sep='\t', encoding='utf-8')

#### Verificando features com dados faltantes

In [8]:
[col for col in df.columns if df[col].isnull().any()]

['cambio', 'portas']

In [9]:
print('Cambio valores distintos:', df.cambio.unique())
print('Valores faltantes: ', df.cambio.isna().sum())
print('\nCambio totais por tipo:')
print(df.cambio.value_counts())

Cambio valores distintos: ['manual' 'automático' nan]
Valores faltantes:  11

Cambio totais por tipo:
manual        1334
automático     366
Name: cambio, dtype: int64


In [10]:
print('Portas valores distintos:', df.portas.unique())
print('Valores faltantes: ', df.portas.isna().sum())
print('\nPortas totais por tipo:')
print(df.portas.value_counts())

Portas valores distintos: [ 3.  2.  4. nan  5.]
Valores faltantes:  225

Portas totais por tipo:
4.0    932
5.0    346
2.0    163
3.0     45
Name: portas, dtype: int64


In [11]:
df['cambio'] = df['cambio'].fillna(method='ffill')
df['portas'] = df['portas'].fillna(method='ffill')

In [12]:
print(df.cambio.value_counts())
print(df.portas.value_counts())

manual        1338
automático     373
Name: cambio, dtype: int64
4.0    1057
5.0     428
2.0     176
3.0      50
Name: portas, dtype: int64


#### Verificando se ficou alguma feature com dados faltantes 

In [13]:
len([col for col in df.columns if df[col].isnull().any()])

0

In [142]:
df.head(20)

,nome,ano,km,cor,cambio,portas,preco,flex
0,Chevrolet Onix 1.0 LT,2020,0.000,Laranja,manual,3.0,59890.0,1.0
1,Fiat Strada Hard Working 1.4,2019,41.070,Branco,manual,2.0,45900.0,1.0
2,Fiat Strada Hard Working 1.4,2019,48.654,Prata,manual,4.0,45900.0,1.0
3,Fiat Strada Hard Working 1.4,2019,19.000,Prata,manual,2.0,45990.0,1.0
4,Fiat Strada Hard Working 1.4,2019,24.715,Branco,manual,2.0,45990.0,1.0
5,Chevrolet Onix 1.0 Turbo Premier,2020,0.000,Azul,automático,2.0,59900.0,0.0
6,Fiat Strada Hard Working 1.4,2019,39.441,Prata,manual,2.0,46000.0,1.0
7,Fiat Strada Hard Working 1.4,2019,18.860,Branco,manual,2.0,46300.0,1.0
8,Fiat Strada Hard Working 1.4,2019,40.856,Prata,manual,2.0,46500.0,1.0
9,Fiat Strada Freedom 1.4,2019,78.000,Vermelho,manual,2.0,46500.0,1.0


In [45]:
X = df.drop(['preco'], axis=1)

In [61]:
X = df.drop(['nome'], axis=1)

In [63]:
y = df.preco

In [64]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [65]:
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

In [66]:
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [67]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [68]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [69]:
numerical_transformer = SimpleImputer(strategy='constant')

In [70]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [71]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [72]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [73]:
from sklearn.metrics import mean_absolute_error

In [74]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

In [75]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['km', 'portas', 'preco']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['cambio'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [76]:
preds = my_pipeline.predict(X_valid)

In [77]:
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 57.74125364431484


In [59]:
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 8657.668681010544


### Aplicando técnica de LabelEncoder

In [12]:
label_encoder = LabelEncoder()

In [19]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['nome', 'ano', 'cor', 'cambio']


In [20]:
y = df.preco

In [21]:
features = df.drop(['preco'], axis=1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.3, random_state=2)

In [27]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_valid], axis=1)

#print("MAE from Approach 3 (One-Hot Encoding):") 
#print(score_dataset(OH_X_train, OH_X_test, y_train, y_valid))

In [28]:
rf = RandomForestRegressor(n_estimators=20, random_state=0)
rf.fit(OH_X_train, y_train)
rf_pred = rf.predict(OH_X_test)

print('RF - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_pred))
print('RF - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, rf_pred))
print('RF - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_pred)))

RF - MAE - Mean Absolute Error: 4444.537615685897
RF - MSE - Mean Squared Error: 51115462.099732764
RF - RMSE - Root Mean Squared Error: 7149.507822202362


In [29]:
dt = DecisionTreeRegressor(max_leaf_nodes=10, random_state=1)
dt.fit(OH_X_train, y_train)
dt_pred = dt.predict(OH_X_test)

print('DT - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, dt_pred))
print('DT - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, dt_pred))
print('DT - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, dt_pred)))

DT - MAE - Mean Absolute Error: 6656.428430480615
DT - MSE - Mean Squared Error: 81941948.2195151
DT - RMSE - Root Mean Squared Error: 9052.179197271511


In [30]:
OH_X_train

,km,portas,0,1,2,3,4,5,6,7,...,204,205,206,207,208,209,210,211,212,213
453,0.000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1554,40.695,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
177,0.000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
786,300.000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1220,100.500,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,47.101,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1608,10.195,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
493,0.000,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
527,0.000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [42]:
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

In [33]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [26]:
#xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.15)

In [27]:
xgbr = xgb.XGBRegressor(verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [36]:
xgbr.fit(label_X_train, y_train)
 
score = xgbr.score(label_X_train, y_train)   
print("Training score: ", score) 

Training score:  0.6988305360104645


In [39]:
# - cross validataion 
scores = cross_val_score(xgbr, label_X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: 0.55


In [40]:
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, label_X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.55


In [43]:
ypred = xgbr.predict(label_X_test)
mse = mean_squared_error(y_test, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse*(1/2.0)))

MSE: 140412403.37
RMSE: 70206201.69


In [48]:
dt = DecisionTreeRegressor(max_leaf_nodes=10, random_state=1)
dt.fit(label_X_train, y_train)
dt_pred = dt.predict(label_X_test)

print('DT - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, dt_pred))
print('DT - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, dt_pred))
print('DT - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, dt_pred)))

DT - MAE - Mean Absolute Error: 7967.64794020087
DT - MSE - Mean Squared Error: 132848193.70879717
DT - RMSE - Root Mean Squared Error: 11525.979078099925
